# Imports

In [ ]:
%load_ext lab_black

import h5py
import os

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm

from typing import Dict, List, Optional, Tuple
import numpy as np
import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    cnn,
)

colors = pc.qualitative.D3
colors_seq = pc.sequential.dense
angstrom = "\u212B"

# Load Data

In [ ]:
ddir = r"E:\atully\arpes_data\2023_February\6eV\TR"
files = ["TR11_Ali_avg.h5"]  # 1.7 eV center energy; -1 to 100 ps

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad_11 = ARPES_DATA[files[0]]

In [ ]:
ddir = r"E:\atully\arpes_data\2023_February\6eV\TR"
files = ["TR3_Ali_avg.h5"]  # 2.15 eV center energy; -1 to 2 ps

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad_3 = ARPES_DATA[files[0]]

In [ ]:
ddir = r"E:\atully\arpes_data\2023_February\6eV\TR"
files = [
    "TR4_Ali_avg.h5"
]  # 2.6 eV center energy; -1 to 1 ps, same number of steps as first 2 ps of TR3

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad_4 = ARPES_DATA[files[0]]

In [ ]:
ddir = r"E:\atully\arpes_data\2023_February\6eV\TR"
files = ["TR_001_1.h5"]  # 1.93 eV center energy; -1 to 100 ps

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad_1 = ARPES_DATA[files[0]]

In [ ]:
ad = ad_4

for k in ["energy", "theta", "phi_or_time"]:
    print(f"{k}.shape = {getattr(ad, k).shape}")
print(f"Data.shape = {ad.data.shape}")

In [ ]:
print(f"Delay range (mm): {np.min(ad.phi_or_time), np.max(ad.phi_or_time)}")
print(
    f"Energy range (eV): {np.round(np.min(ad.energy), 2), np.round(np.max(ad.energy), 2)}"
)
print(f"Theta range: {np.round(np.min(ad.theta), 1), np.round(np.max(ad.theta), 1)}")

# Build Dataset

## Datasets
ad_11 --> 1.7 eV center energy; -1 to 100 ps; 80 steps in delay (mm), but -1 to 1 ps in 21 steps (not great time resolution...)

ad_1 --> 1.93 eV center energy; -1 to 100 ps; 80 steps in delay (mm), but -1 to 1 ps in 21 steps (not great time resolution...)

ad_3 --> 2.15 eV center energy; -1 to 2 ps; 62 steps in delay (mm)

ad_4 --> 2.6 eV center energy; -1 to 1 ps; 42 steps in delay (mm)

In [ ]:
## Zero Delay ##

time_zero = 37.958  # from BiSe

## HOMO is at 2.05 eV below EF, based on fits from this data averaged with fits from tr-ARPES results ##

EF_400 = 1.91  # in kinetic energy, slit 400
EF_700 = 1.94  # in kinetic energy, slit 700

homo = -2.05

homo_400 = homo + EF_400
homo_700 = homo + EF_700

In [ ]:
## Integrate over desired angular range ##

slice_dim = "x"
slice_val = 0
int_range = 20  # if this value is more that the integration range, my get_2D_slice function will just integrate over the max range.

xlim = None
ylim = None
x_bin = 1
y_bin = 1

In [ ]:
all_vals = []
for ad in [ad_11, ad_3, ad_4, ad_1]:
    all_vals.append(
        tr_functions.slice_datacube(
            ad_dataclass=ad,
            slice_dim=slice_dim,
            slice_val=slice_val,
            int_range=int_range,
            xlim=xlim,
            ylim=(
                ad.energy[57],
                ad.energy[1007],
            ),  # get rid of zero padding on datasets
            x_bin=x_bin,
            y_bin=y_bin,
            norm_data=False,
            plot_data=False,
        )
    )
x_11, y_11, d_11 = all_vals[0]
x_3, y_3, d_3 = all_vals[1]
x_4, y_4, d_4 = all_vals[2]
x_1, y_1, d_1 = all_vals[3]

In [ ]:
## Plot Data: MPL ##

fig, ax = plt.subplots(1)

ax.pcolormesh(x_11, y_11, d_11, shading="auto", cmap="plasma", vmin=0, vmax=2.5)
ax.pcolormesh(x_3, y_3, d_3, shading="auto", cmap="plasma", vmin=0, vmax=0.2)
ax.pcolormesh(x_4, y_4, d_4, shading="auto", cmap="plasma", vmin=0, vmax=0.1)

ax.set_xlim(xmin=37.81, xmax=38.11)

# plt.save_fig(r'C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\TR3&TR4&T11_plasma_mpl.png')

In [ ]:
## Normalize plots relative to backgrounds ##

# TR4
xlim = (37.81, 37.84)
ylim = (2.3, 2.9)

tr4_bg = tr_functions.get_avg_background(x_4, y_4, d_4, xlim, ylim)
# tr4_bg = get_avg_background(x_4, y_4, d4_norm, xlim, ylim)  # check

# TR3
xlim = (37.81, 37.84)
ylim = (2.15, 2.48)

tr3_bg = tr_functions.get_avg_background(x_3, y_3, d_3, xlim, ylim)

# TR11
xlim = (37.81, 37.85)
ylim = (1.93, 2.04)

tr11_bg = tr_functions.get_avg_background(x_11, y_11, d_11, xlim, ylim)
# tr11_bg = get_avg_background(x_11, y_11, d11_norm, xlim, ylim)  # check

tr4_bg, tr3_bg, tr11_bg

In [ ]:
norm_tr4_to_tr3 = tr3_bg / tr4_bg
norm_tr11_to_tr3 = tr3_bg / tr11_bg

d4_norm = d_4 * norm_tr4_to_tr3
d11_norm = d_11 * norm_tr11_to_tr3

In [ ]:
## give data equivalent x axes ##

xlim = (37.81, 38.11)

x4, y4, d4 = analysis_functions.limit_dataset(x_4, y_4, d4_norm, xlim=xlim, ylim=None)
x3, y3, d3 = analysis_functions.limit_dataset(x_3, y_3, d_3, xlim=xlim, ylim=None)
x11, y11, d11 = analysis_functions.limit_dataset(
    x_11, y_11, d11_norm, xlim=xlim, ylim=None
)

x1, y1, d1 = x_1, y_1, d_1

In [ ]:
## Adjust energy axis to be relative to HOMO ##

homo_zero = False
homo_zero = True

if homo_zero:
    y4 = (
        y4 - homo_400
    )  # homo is negative, so energy scale will increase, because we're referencing a negative number rather than zero
    y3 = y3 - homo_700
    y11 = y11 - homo_400
    y1 = y1 - homo_700

In [ ]:
## Linearly interpolate x11 to match resolution of TR3 and TR4 ##

x, y, d = x11, y11, d11

new_d = tr_functions.interpolate_dataset(x, y, d, xref=x3)

fig = tr_functions.default_fig()
fig.add_trace(go.Heatmap(x=x3, y=y, z=new_d))
fig.show()

print(new_d.shape)

In [ ]:
fig = tr_functions.default_fig()
fig.add_trace(go.Heatmap(x=x3, y=y3, z=d3, coloraxis="coloraxis"))
fig.update_coloraxes(cmin=0, cmax=0.2)
fig.show()

In [ ]:
fig = tr_functions.default_fig()
fig.add_trace(go.Heatmap(x=x4, y=y4, z=d4))
fig.show()

In [ ]:
## Stitch Data ##

## TR4 & TR3
x_s1, y_s1, data_s1 = tr_functions.stitch_2_datasets(
    d4, x4, y4, d3, x3, y3, stitch_dim="y"
)

## TR4 & TR3 & TR11
x_s2, y_s2, data_s2 = tr_functions.stitch_2_datasets(
    new_d,
    x3,
    y11,
    data_s1,
    x_s1,
    y_s1,
    stitch_dim="y",
)

## TR11 & TR3
x_s3, y_s3, data_s3 = tr_functions.stitch_2_datasets(
    new_d, x3, y11, d3, x3, y3, stitch_dim="y"
)

In [ ]:
## Plot Stitched Data ##

In [ ]:
yaxis_title = "E - E<sub>HOMO</sub> [eV]"
xaxis_title = "Delay [ps]"

In [ ]:
## Plot TR3 & TR4 ##
x_plot, y_plot, z_plot = x_s1, y_s1, data_s1

## toggle_time ?
toggle_time = "picoseconds"
# toggle_time = "mm"

## Logplot?
logplot = False
# logplot = True

if logplot:
    z_plot = np.log(z_plot)
    title = f"TR3 & TR4 (logplot)"
else:
    title = f"TR3 & TR4"

## Convert mm to ps
if toggle_time == "picoseconds":
    x_plot = tr_functions.mm_to_ps(x_plot, time_zero)

## Plot Data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
)
fig.add_trace(
    go.Heatmap(x=x_plot, y=y_plot, z=z_plot, coloraxis="coloraxis")
    # np.log(data)
)

fig.update_coloraxes(cmin=0, cmax=0.4)
# fig.update_coloraxes(colorscale="hot", cmin=None, cmax=0.3, reversescale=True)
fig.update_layout(width=800, height=600)
fig.show()

# fig.write_image(r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\TR3&TR4.png")

In [ ]:
## Plot TR11 & TR3 & TR4 ##
x_plot, y_plot, z_plot = x_s2, y_s2, data_s2

## toggle_time ?
toggle_time = "picoseconds"
# toggle_time = "mm"

## Logplot?
logplot = False
# logplot = True

if logplot:
    z_plot = np.log(z_plot)
    title = f"TR11 & TR3 & TR4 (logplot)"
else:
    title = f"TR11 & TR3 & TR4"

## Convert mm to ps
if toggle_time == "picoseconds":
    x_plot = tr_functions.mm_to_ps(x_plot, time_zero)

## Plot Data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    dtick_y=0.4,
)
fig.add_trace(
    go.Heatmap(x=x_plot, y=y_plot, z=z_plot, coloraxis="coloraxis")
    # np.log(data)
)

# fig.update_coloraxes(colorscale="hot", cmin=0, cmax=0.3, reversescale=True)
fig.update_layout(width=800, height=600)
fig.show()

# fig.write_image(r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\TR11&TR3&TR4.png")

In [ ]:
## Plot TR11 & TR3 ##
x_plot, y_plot, z_plot = x_s3, y_s3, data_s3

## toggle_time ?
toggle_time = "picoseconds"
# toggle_time = "mm"

## Logplot?
logplot = False
# logplot = True

if logplot:
    z_plot = np.log(z_plot)
    title = f"TR11 & TR3 (logplot)"
else:
    title = f"TR11 & TR3"

## Convert mm to ps
if toggle_time == "picoseconds":
    x_plot = tr_functions.mm_to_ps(x_plot, time_zero)

## Plot Data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
)
fig.add_trace(go.Heatmap(x=x_plot, y=y_plot, z=z_plot, coloraxis="coloraxis"))

fig.update_coloraxes(cmin=0, cmax=None)
fig.update_layout(width=800, height=600)
fig.show()

# fig.write_image(r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\TR11&TR3.png")

In [ ]:
## Plot TR1 ##
x_plot, y_plot, z_plot = x_1, y_1, analysis_functions.norm_data(d_1)

## toggle_time ?
toggle_time = "picoseconds"
# toggle_time = "mm"

## Logplot?
logplot = False
# logplot = True

if logplot:
    z_plot = np.log(z_plot)
    title = f"TR1 (logplot)"
else:
    title = f"TR1"

## Convert mm to ps
if toggle_time == "picoseconds":
    x_plot = tr_functions.mm_to_ps(x_plot, time_zero)

## Plot Data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    dtick_y=0.2,
)
fig.add_trace(
    go.Heatmap(x=x_plot, y=y_plot, z=z_plot, coloraxis="coloraxis")
    # np.log(data)
)

# fig.update_coloraxes(colorscale="hot", cmin=0, cmax=0.3, reversescale=True)
fig.update_layout(width=800, height=600)
fig.show()

# fig.write_image(r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\TR11&TR3&TR4.png")

# New Functions

In [ ]:
def fit_FD_7gauss(
    x,
    data,
    offset_type="constant",
    params=None,
    collect_centers=False,
    fix_centers=False,
    centers=None,
    plot_fit=True,
):
    if fix_centers and not centers:
        raise ValueError(
            f"fix_centers is {fix_centers} and centers is {centers}. If fix_centers is True, must specify list of center values."
        )

    ## FD
    def fermi_dirac(x, center, theta, amp):
        arg = (x - center) / (2 * theta)  # x=E, center=mu, theta = k_B * T
        return -amp / 2 * np.tanh(arg)

    ## Offset
    c = np.mean(data)
    b = (data[-1] - data[0]) / (x[-1] - x[0])
    a = 0

    offset = fitting_functions.offset_model(offset_type, a, b, c)

    ## Gaussians
    gauss1 = fitting_functions.make_gaussian(
        num="A_", amplitude=1, center=1.65, sigma=0.1
    )
    gauss2 = fitting_functions.make_gaussian(
        num="B_", amplitude=1, center=1.8, sigma=0.1
    )
    gauss3 = fitting_functions.make_gaussian(
        num="C_", amplitude=1, center=1.95, sigma=0.1
    )
    gauss4 = fitting_functions.make_gaussian(
        num="D_", amplitude=1, center=2.15, sigma=0.1
    )
    gauss5 = fitting_functions.make_gaussian(
        num="E_", amplitude=0.5, center=2.4, sigma=0.1
    )
    gauss6 = fitting_functions.make_gaussian(
        num="F_", amplitude=0.5, center=2.6, sigma=0.1
    )
    gauss7 = fitting_functions.make_gaussian(
        num="G_", amplitude=0.5, center=2.7, sigma=0.1
    )

    ## Full model
    full_model = (
        lm.models.Model(fermi_dirac)
        + gauss1
        + gauss2
        + gauss3
        + gauss4
        + gauss5
        + gauss6
        + gauss7
        + offset
    )

    ## Run model with linear params starting values, force constant offset
    if offset_type == "constant":
        # params = linear_params.copy()
        params = params
        params["b"].value = 0
        params["b"].vary = False

    params = full_model.make_params()

    params["center"].value = 2.025
    params["center"].vary = False
    params["theta"].value = k_B * (10.6)
    params["theta"].min = 0
    params["amp"].value = 1
    params["amp"].min = 0

    params["iA__center"].min = 1.5
    params["iA__center"].max = 1.7
    params["iB__center"].min = 1.7
    params["iB__center"].max = 1.9
    params["iC__center"].min = 1.9
    params["iC__center"].max = 2.1
    params["iD__center"].min = 2.0
    params["iD__center"].max = 2.2

    if fix_centers:
        for letter, fixed in zip("ABCDEFG", centers):
            params[f"i{letter}__center"].value = fixed
            params[f"i{letter}__center"].vary = False

    fit = full_model.fit(data, x=x, params=params)
    if plot_fit:
        fit.plot()

    if offset_type == "linear":
        linear_params = fit.params.copy()

        return fit, linear_params

    ## collect fixed center values
    if collect_centers:
        centers_fixed = []
        components = fit.eval_components(x=y_1d)
        for model_name, model_value in list(components.items())[1:8]:
            centers_fixed.append(
                fit.params[f"{model_name}center"].value.astype("float32")
            )

        return fit, centers_fixed

    else:
        return fit

# Analysis

In [ ]:
T = 10.6  # measurement temp

k_B = 8.617333e-5  # eV/K

In [ ]:
## Plot of TR1##
title = "TR1"
x, y, d = x1, y1, analysis_functions.norm_data(d1)

# Plot Data
fig = tr_functions.thesis_fig(
    title=f"{title}",
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    gridlines=False,
    height=400,
)

x_plot = tr_functions.mm_to_ps(x, time_zero)
y_plot = y

fig.add_trace(go.Heatmap(x=x_plot, y=y_plot, z=d, coloraxis="coloraxis"))

# fig.update_coloraxes(colorscale="RdBu", cmid=0, showscale=True)

In [ ]:
## Difference Map  TR1##
title = "Difference Map of TR1"
x, y, d = x1, y1, d1

# d_diff = d_2d - d_2d[:, 2][:, None]
d_diff_1 = d - np.mean(d[:, 0:4], axis=1)[:, None]

# Plot Data
fig = tr_functions.thesis_fig(
    title=f"{title}",
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    gridlines=False,
    height=400,
)

x_plot = tr_functions.mm_to_ps(x, time_zero)
y_plot = y

fig.add_trace(go.Heatmap(x=x_plot, y=y_plot, z=d_diff_1, coloraxis="coloraxis"))
# for h in [1.63, 1.8, 1.98]:
#     fig.add_hline(y=h, line=dict(color="black", width=1, dash="dash"))
fig.update_coloraxes(colorscale="RdBu", cmid=0, showscale=True)

In [ ]:
## EDCs with different time integration ##

title = "TR1"
spacing = 0

# x_plot, y_plot, z_plot = x1, y1, analysis_functions.norm_data(d1)  # regular
x_plot, y_plot, z_plot = x1, y1, d_diff_1  # difference plot

if np.min(x_plot) > 0:
    x_edc = tr_functions.mm_to_ps(x_plot, time_zero)


## Logplot?
logplot = False
# logplot = True

if logplot:
    # z_plot = np.log(z_plot)
    title = f"{title}<br>(logplot)"
else:
    title = f"{title}"


## Set up integration limits
# xlim1 = (0, 100)
# xlim1 = (0, 5)
# xlims = [xlim1]

xlim1 = (-1, -0.5)  # negative delay
xlim2 = (-0.5, 0.5)  # zero delay
xlim3 = (1, 3)
xlim4 = (1, 10)
xlim5 = (10, 20)
xlim6 = (20, 40)
xlim7 = (40, 60)
xlim8 = (60, 80)
xlim9 = (80, 100)
xlim10 = (0, 100)


xlims = [xlim1, xlim2, xlim3, xlim4, xlim5, xlim6, xlim7, xlim8, xlim9, xlim10]

ylim = None

# Get and plot data

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=yaxis_title,
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
)

for i, xlim in enumerate(xlims):
    color = colors[i % len(colors)]

    y_1d, col = tr_functions.get_1d_x_slice(
        x=x_edc,
        y=y_plot,
        data=z_plot,
        ylims=ylim,
        x_range=xlim,
    )

    if logplot:
        col = np.log(col)

    # Plot Data
    fig.add_trace(
        go.Scatter(
            x=y_1d,
            y=i * spacing + col,
            name=f"{np.round(xlim[0], 1), np.round(xlim[1], 2)} ps",
            line=dict(color=color),
        )
    )

fig.show()

# fig.write_image(
#     r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\TR11&TR3&TR4_EDC.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\TR11&TR3&TR4_EDC_logplot.png"
# )

In [ ]:
x_plot, y_plot, z_plot = x1, y1, analysis_functions.norm_data(d1)
spacing = 0.2
title = "Waterfall Plot of TR1"

# # Difference Map
# x_plot, y_plot, z_plot = (
#     x1,
#     y1,
#     analysis_functions.norm_data(d_diff_1),
# )  # difference plot
# title = f"{title} Diff Map"

if np.min(x_plot) > 0:
    x_edc = tr_functions.mm_to_ps(x_plot, time_zero)

slice_window = 20
steps = 20
new_axes = []
new_data = []
fits = []
fit_components = []
ylim = None
xlim = (np.min(x_edc), np.max(x_edc))
title = f"{title} ({slice_window} ps window)"

print(f"Minimum window size: {np.max(np.diff(x_edc))}")

fig = tr_functions.thesis_fig(
    title=title,
    yaxis_title=yaxis_title,
    xaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
    height=400,
)

for i, xval in enumerate(np.linspace(xlim[0], xlim[1], steps)):
    x_window = (xval - slice_window / 2, xval + slice_window / 2)
    # print(x_window)
    # if x_window[1] - x_window[0] <= np.max(
    #     np.diff(x_edc)
    # ):  # gives resolution = minimum window size
    #     raise ValueError(f"{x_window} is <= min window size ({np.max(np.diff(x_edc))})")
    y_1d, col = tr_functions.get_1d_x_slice(
        x=x_edc,
        y=y_plot,
        data=z_plot,
        ylims=ylim,
        x_range=x_window,
    )

    fig.add_trace(
        go.Scatter(
            y=y_1d,
            x=i * spacing + col,
            # x=xval + col * 20,
            name=f"{np.round(x_window[0], 1), np.round(x_window[1], 1)} ps",
            # line=dict(color=colors[i]),
            # line=dict(color=colors_seq[i + 1]),
            line=dict(color=colors[0]),
        )
    )
    new_axes.append(y_1d)
    new_data.append(col)

    x, data = y_1d, col
    offset_type = "constant"

    ## FD
    def fermi_dirac(x, center, theta, amp):
        arg = (x - center) / (2 * theta)  # x=E, center=mu, theta = k_B * T
        return -amp / 2 * np.tanh(arg)

    ## Offset
    c = np.mean(data)
    b = (data[-1] - data[0]) / (x[-1] - x[0])
    a = 0

    offset = fitting_functions.offset_model(offset_type, a, b, c)

    # Fit for 2 peaks
    gauss1 = fitting_functions.make_gaussian(
        num="A_", amplitude=1, center=1.8, sigma=0.1
    )  # 1.86
    gauss2 = fitting_functions.make_gaussian(
        num="B_", amplitude=1, center=2.1, sigma=0.1
    )  # 2.1

    ## Full model
    full_model = lm.models.Model(fermi_dirac) + gauss1 + gauss2 + offset

    params = full_model.make_params()

    if offset_type == "constant":
        # params = linear_params.copy()
        params = params
        params["b"].value = 0
        params["b"].vary = False

    params[
        "center"
    ].value = 2.013  # determined by fitting from (0, 100) ps with a constant offset
    params["center"].vary = False
    # # params["c"].value = 0  # determined by fitting from (0, 100) ps with a constant offset
    # # params["c"].vary = False
    params["theta"].value = k_B * (10.6)
    params["theta"].min = 0
    params["amp"].value = 1
    params["amp"].min = 0

    params["iA__sigma"].value = tr_functions.fwhm_to_sig(0.2)
    # params["iA__sigma"].vary = False
    params["iB__sigma"].value = tr_functions.fwhm_to_sig(0.1)
    params["iB__sigma"].max = tr_functions.fwhm_to_sig(0.2)

    fit = full_model.fit(data, x=x, params=params)
    fits.append(fit)
    fig.add_trace(
        go.Scatter(
            x=i * spacing + fit.eval(x=y_1d),
            y=y_1d,
            name="fit",
            line=dict(color="red"),
            opacity=0.5,
        )
    )

    components = fit.eval_components(x=y_1d)
    fit_components.append(components)

#     if plot_fit:
#         fit.plot()

#     if offset_type == "linear":
#         linear_params = fit.params.copy()

#     ## collect fixed center values
#     if collect_centers:
#         centers_fixed = []
#         components = fit.eval_components(x=y_1d)
#         for model_name, model_value in list(components.items())[1:8]:
#             centers_fixed.append(fit.params[f"{model_name}center"].value.astype("float32"))


fig.show()

In [ ]:
fits[-3].plot()

In [ ]:
fig = tr_functions.thesis_fig(
    title=f"{title}<br> Fit Components",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

fit = fits[-3]

for model_name, model_value in fit_components[-3].items():
    fig.add_trace(
        go.Scatter(
            x=y_1d,
            y=model_value,
            name=model_name,
        )
    )


fig.data[3].update(name="offset")

for model_name, model_value in list(components.items())[1:3]:
    fig.add_annotation(
        x=fit.params[f"{model_name}center"].value,
        yref="y domain",
        y=0.3,
        showarrow=False,
        text=f'{fit.params[f"{model_name}center"].value:.2f} eV<br>{fit.params[f"{model_name}fwhm"].value:.2f} eV',
        font=dict(size=20),
        bgcolor="white",
    )


fig.show()

In [ ]:
# xs = [np.mean(vs) for vs in [xlim1, xlim2, xlim3, xlim4]]
xs = np.linspace(xlim[0], xlim[1], steps)
colors = pc.qualitative.D3

for param_name in ["amplitude", "height", "fwhm", "center"]:
    # Fig for each fitting parameter
    fig = tr_functions.default_fig()
    fig.update_layout(
        title=param_name,
        xaxis_title="delay (ps)",
        yaxis_title=param_name,
        font=dict(size=20),
    )

    # For each peak individually (one trace per peak)
    for i, letter in enumerate("AB"):
        color = colors[i % len(colors)]
        ys = []
        y_errs = []
        y_inits = []
        for fit in fits:
            par = fit.params[f"i{letter}__{param_name}"]
            ys.append(par.value)
            y_errs.append(par.stderr)
            y_inits.append(par.init_value)

        fig.add_trace(
            go.Scatter(
                x=xs,
                y=ys,
                error_y=dict(array=y_errs),
                name="value",
                line=dict(color=color),
                legendgroup=letter,
                legendgrouptitle_text=letter,
                mode="markers+lines",
            )
        )
        # fig.add_trace(
        #     go.Scatter(
        #         x=xs,
        #         y=y_inits,
        #         name="inits",
        #         line=dict(color=color, dash="dash"),
        #         opacity=0.6,
        #         legendgroup=letter,
        #     )
        # )
    fig.show()

In [ ]:
x_plot, y_plot, z_plot = x1, y1, analysis_functions.norm_data(d1)
spacing = 0.2
title = "Waterfall Plot of TR1"

# # Difference Map
# x_plot, y_plot, z_plot = (
#     x1,
#     y1,
#     analysis_functions.norm_data(d_diff_1),
# )  # difference plot
# title = f"{title} Diff Map"

if np.min(x_plot) > 0:
    x_edc = tr_functions.mm_to_ps(x_plot, time_zero)

slice_window = 20
steps = 10
new_axes = []
new_data = []
ylim = None
xlim = (np.min(x_edc), np.max(x_edc))
title = f"{title} ({slice_window} ps window)"

print(f"Minimum window size: {np.max(np.diff(x_edc))}")

fig = tr_functions.thesis_fig(
    title=title,
    yaxis_title=yaxis_title,
    xaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
    height=400,
)

for i, xval in enumerate(np.linspace(xlim[0], xlim[1], steps)):
    x_window = (xval - slice_window / 2, xval + slice_window / 2)
    
    y_1d, col = tr_functions.get_1d_x_slice(
        x=x_edc,
        y=y_plot,
        data=z_plot,
        ylims=ylim,
        x_range=x_window,
    )

    fig.add_trace(
        go.Scatter(
            y=y_1d,
            x=i * spacing + col,
            # x=xval + col * 20,
            name=f"{np.round(x_window[0], 1), np.round(x_window[1], 1)} ps",
            # line=dict(color=colors[i]),
            line=dict(color=colors_seq[i + 1]),
            # line=dict(color=color),
        )
    )
    new_axes.extend(y_1d)
    new_data.extend(col)
fig.show()

# Evolution of Peaks for TR11, TR3, & TR4

In [ ]:
## EDCs with different time integration ##

title = "TR11 & TR3 & TR4"

x_plot, y_plot, z_plot = x_s2, y_s2, data_s2

if np.min(x_plot) > 0:
    x_edc = tr_functions.mm_to_ps(x_plot, time_zero)


## Logplot?
logplot = False
# logplot = True

if logplot:
    # z_plot = np.log(z_plot)
    title = f"{title}<br>(logplot)"
else:
    title = f"{title}"


## Set up integration limits
xlim1 = (
    np.min(x_edc),
    -0.5,
)  # purely negative delay (taking into account time overlap pulsewidth)
# xlim2 = (np.min(x_edc), 0)  # negative delay
# xlim3 = (
#     -0.25,
#     1,
# )  # -0.25 to 1 ps positive delay, to help account for width of time overlap pulse (~ 500 fs)
# xlim4 = (0, 1)  # zero to 1 ps positive delay

xlim2 = (-0.5, 0.5)  # zero delay
xlim3 = (0.5, 1)  # longer pumped signal

# xlims = [xlim1, xlim2, xlim3, xlim4]
xlims = [xlim1, xlim2, xlim3]

ylim = None

# Get and plot data

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=yaxis_title,
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

for i, xlim in enumerate(xlims):
    color = colors[i % len(colors)]

    y_1d, col = tr_functions.get_1d_x_slice(
        x=x_edc,
        y=y_plot,
        data=z_plot,
        ylims=ylim,
        x_range=xlim,
    )

    if logplot:
        col = np.log(col)

    # Plot Data
    fig.add_trace(
        go.Scatter(
            x=y_1d,
            y=col,
            name=f"{np.round(xlim[0], 1), np.round(xlim[1], 2)} ps",
            line=dict(color=color),
        )
    )

fig.show()

# fig.write_image(
#     r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\TR11&TR3&TR4_EDC.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\TR11&TR3&TR4_EDC_logplot.png"
# )

In [ ]:
## EDCs with different time integration ##

title = "TR11 & TR3"

x_plot, y_plot, z_plot = x_s3, y_s3, data_s3

if np.min(x_plot) > 0:
    x_edc = tr_functions.mm_to_ps(x_plot, time_zero)


## Logplot?
logplot = False
# logplot = True

if logplot:
    # z_plot = np.log(z_plot)
    title = f"{title}<br>(logplot)"
else:
    title = f"{title}"


## Set up integration limits
xlim1 = (
    np.round(np.min(x_edc), 1),
    -0.5,
)  # purely negative delay (taking into account time overlap pulsewidth)

# xlim2 = (np.min(x_edc), 0)  # negative delay
# xlim3 = (
#     -0.25,
#     1,
# )  # -0.25 to 1 ps positive delay, to help account for width of time overlap pulse (~ 500 fs)
# xlim4 = (0, 1)  # zero to 1 ps positive delay

xlim2 = (-0.5, 0.5)  # zero delay
xlim3 = (0.5, 1)  # longer pumped signal

# xlims = [xlim1, xlim2, xlim3, xlim4]
xlims = [xlim1, xlim2, xlim3]
ylim = (np.min(y_plot), 2.4)
# ylim = None

## Get and plot data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=yaxis_title,
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
    dtick_y=1,
)


for i, xlim in enumerate(xlims):
    color = colors[i % len(colors)]

    y_1d, col = tr_functions.get_1d_x_slice(
        x=x_edc,
        y=y_plot,
        data=z_plot,
        ylims=ylim,
        x_range=xlim,
    )

    if logplot:
        col = np.log(col)

    fig.add_trace(
        go.Scatter(
            x=y_1d,
            y=col,
            name=f"{np.round(xlim[0], 2), np.round(xlim[1], 2)} ps",
            line=dict(color=color),
        )
    )

fig.show()

# fig.write_image(
#     r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\TR11&TR3_EDC.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\TR11&TR3_EDC_logplot.png"
# )

# Fitting Peaks

In [ ]:
## EDCs with different time integration ##

title = "TR11 & TR3 & TR4"
x_plot, y_plot, z_plot = x_s2, y_s2, data_s2
# xlim = (-0.75, -0.5)
# xlim = (-0.25, 0.25)
xlim = (0.25, 0.5)
# xlim = (0.5, 0.75)
# xlim = (0.75, 1)
ylim = None

logplot = False

if np.min(x_plot) > 0:
    x_edc = tr_functions.mm_to_ps(x_plot, time_zero)

# Get and plot data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=yaxis_title,
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=x_edc,
    y=y_plot,
    data=z_plot,
    ylims=ylim,
    x_range=xlim,
)

if logplot:
    col = np.log(col)

fig.add_trace(
    go.Scatter(
        x=y_1d,
        y=col,
        name=f"{np.round(xlim[0], 2), np.round(xlim[1], 2)} ps",
        line=dict(color=colors[0]),
    )
)

fig.update_layout(showlegend=True)

fig.show()

In [ ]:
# ## Fit ##

# ## STEPS:
# # 1. run with offset_type = "linear", fix_centers=False, and collect_centers=False
# # 2. run with offset_type = "constant"
# # 3. once the center values of the fit seem reasonable and robust, re-run with offset_type = "constant" and collect_centers=True
# # 4. can then run for other delay integration regions with fix_centers=True
# # 5. once you know the model and have the centers fixed, can run local fit_FD_#gauss function

# x = y_1d
# data = col

# offset_type = "linear"
# offset_type = "constant"
# plot_fit = True
# fix_centers = False
# fix_centers = True
# collect_centers = False
# # collect_centers = True


# ## FD
# def fermi_dirac(x, center, theta, amp):
#     arg = (x - center) / (2 * theta)  # x=E, center=mu, theta = k_B * T
#     return -amp / 2 * np.tanh(arg)


# ## Offset
# c = np.mean(data)
# b = (data[-1] - data[0]) / (x[-1] - x[0])
# a = 0

# offset = fitting_functions.offset_model(offset_type, a, b, c)

# ## Gaussians
# gauss1 = fitting_functions.make_gaussian(num="A_", amplitude=1, center=1.65, sigma=0.1)
# gauss2 = fitting_functions.make_gaussian(num="B_", amplitude=1, center=1.8, sigma=0.1)
# gauss3 = fitting_functions.make_gaussian(num="C_", amplitude=1, center=1.95, sigma=0.1)
# gauss4 = fitting_functions.make_gaussian(num="D_", amplitude=1, center=2.15, sigma=0.1)
# gauss5 = fitting_functions.make_gaussian(num="E_", amplitude=0.5, center=2.4, sigma=0.1)
# gauss6 = fitting_functions.make_gaussian(num="F_", amplitude=0.5, center=2.6, sigma=0.1)
# gauss7 = fitting_functions.make_gaussian(num="G_", amplitude=0.5, center=2.7, sigma=0.1)


# ## Full model
# full_model = (
#     lm.models.Model(fermi_dirac)
#     + gauss1
#     + gauss2
#     + gauss3
#     + gauss4
#     + gauss5
#     + gauss6
#     + gauss7
#     + offset
# )


# ## Run model with linear params starting values, force constant offset
# if offset_type == "constant":
#     params = linear_params.copy()
#     params["b"].value = 0
#     params["b"].vary = False

# params = full_model.make_params()

# params["center"].value = 2.025
# params["center"].vary = False
# params["theta"].value = k_B * (10.6)
# params["theta"].min = 0
# params["amp"].value = 1
# params["amp"].min = 0

# params["iA__center"].min = 1.5
# params["iA__center"].max = 1.7
# params["iB__center"].min = 1.7
# params["iB__center"].max = 1.9
# params["iC__center"].min = 1.9
# params["iC__center"].max = 2.1
# params["iD__center"].min = 2.0
# params["iD__center"].max = 2.2

# if fix_centers:
#     for letter, fixed in zip("ABCDEFG", centers_fixed):
#         params[f"i{letter}__center"].value = fixed
#         params[f"i{letter}__center"].vary = False


# fit = full_model.fit(data, x=x, params=params)
# if plot_fit:
#     fit.plot()

# # method="powell"
# if offset_type == "linear":
#     linear_params = fit.params.copy()

# ## collect fixed center values
# if collect_centers:
#     centers_fixed = []
#     components = fit.eval_components(x=y_1d)
#     for model_name, model_value in list(components.items())[1:8]:
#         centers_fixed.append(fit.params[f"{model_name}center"].value.astype("float32"))

In [ ]:
## Step 1:
# fit, linear_params = fit_FD_7gauss(
#     x=y_1d,
#     data=col,
#     offset_type="linear",
#     params=None,
#     collect_centers=False,
#     fix_centers=False,
#     centers=None,
#     plot_fit=True,
# )

## Step 2:
# fit = fit_FD_7gauss(
#     x=y_1d,
#     data=col,
#     offset_type="constant",
#     params=linear_params,
#     collect_centers=False,
#     fix_centers=False,
#     centers=None,
#     plot_fit=True,
# )

## Step 3:
# fit, centers = fit_FD_7gauss(
#     x=y_1d,
#     data=col,
#     offset_type="constant",
#     params=linear_params,
#     collect_centers=True,
#     fix_centers=False,
#     centers=None,
#     plot_fit=True,
# )

In [ ]:
centers_fixed

In [ ]:
centers

In [ ]:
## Plot Fit Components ##

fig = tr_functions.thesis_fig(
    title=f"{title}<br> Fit Components",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

components = fit.eval_components(x=y_1d)
for model_name, model_value in components.items():
    fig.add_trace(
        go.Scatter(
            x=y_1d,
            y=model_value,
            name=model_name,
        )
    )

fig.data[8].update(name="offset")

for model_name, model_value in list(components.items())[1:8]:
    fig.add_annotation(
        x=fit.params[f"{model_name}center"].value,
        # y=fit.eval(x=fit.params[f"{model_name}center"].value),
        # xref="x domain",
        yref="y domain",
        # The arrow head will be 25% along the x axis, starting from the left
        # x=0.25,
        # The arrow head will be 40% along the y axis, starting from the bottom
        y=0.3,
        showarrow=False,
        text=f'{fit.params[f"{model_name}center"].value:.2f} eV<br>{fit.params[f"{model_name}fwhm"].value:.2f} eV',
        font=dict(size=12),
        # ax=-300,
        # ay=0,
        bgcolor="white",
        # opacity=1,
        # bordercolor=colors[0],
        # borderwidth=2,
        # borderpad=4,
    )


fig.show()

In [ ]:
## Plot Data and Fit
fig = tr_functions.thesis_fig(
    title=f"{title}",
    xaxis_title=yaxis_title,
    yaxis_title="Residuals",
    equiv_axes=False,
    gridlines=False,
    height=400,
    width=900,
)

fig.add_trace(go.Scatter(x=y_1d, y=col - fit.eval(x=y_1d), name="fit"))
fig.add_hline(y=0, line=dict(dash="dash"))

In [ ]:
## Plot Data and Fit
fig = tr_functions.thesis_fig(
    title=f"{title}",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=x_edc,
    y=y_plot,
    data=z_plot,
    ylims=ylim,
    x_range=xlim,
)

fig.add_trace(
    go.Scatter(
        x=y_1d,
        y=col,
        name=f"{np.round(xlim[0], 2), np.round(xlim[1], 2)} ps",
        line=dict(color=colors[0]),
    )
)

fig.add_trace(go.Scatter(x=y_1d, y=fit.eval(x=y_1d), name="fit"))

fig.update_layout(showlegend=True)

for model_name, model_value in list(components.items())[1:8]:
    fig.add_annotation(
        x=fit.params[f"{model_name}center"].value,
        y=fit.eval(x=fit.params[f"{model_name}center"].value),
        text=f'{fit.params[f"{model_name}center"].value:.2f} ',
        font=dict(size=18, color=colors[0]),
        # ax=-300,
        # ay=0,
        bgcolor="white",
        # opacity=1,
        # bordercolor=colors[0],
        # borderwidth=2,
        # borderpad=4,
    )

fig.show()

# print(f"Center FD = {fit.params['center'].value:.2f} eV")
# print(f"T = {fit.params['theta'].value / k_B:.2f} K")
# print(f"Center A = {fit.params['iA__center'].value:.2f} eV")
# print(f"FWHM A = {fit.params['iA__fwhm'].value:.3f} ps")
# print(f"Center B = {fit.params['iB__center'].value:.2f} eV")
# print(f"FWHM B = {fit.params['iB__fwhm'].value:.3f} ps")
# print(f"Center C = {fit.params['iC__center'].value:.2f} eV")
# print(f"FWHM C = {fit.params['iC__fwhm'].value:.3f} ps")
# print(f"Center D = {fit.params['iD__center'].value:.2f} eV")
# print(f"FWHM D = {fit.params['iD__fwhm'].value:.3f} ps")
# print(f"Center E = {fit.params['iE__center'].value:.2f} eV")
# print(f"FWHM E = {fit.params['iE__fwhm'].value:.3f} ps")
# print(f"Center F = {fit.params['iF__center'].value:.2f} eV")
# print(f"FWHM F = {fit.params['iF__fwhm'].value:.3f} ps")
# print(f"Center G = {fit.params['iG__center'].value:.2f} eV")
# print(f"FWHM G = {fit.params['iG__fwhm'].value:.3f} ps")

In [ ]:
## Plot Data and Fit
fig = tr_functions.thesis_fig(
    title=f"{title} (logplot)",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=x_edc,
    y=y_plot,
    data=z_plot,
    ylims=ylim,
    x_range=xlim,
)

fig.add_trace(
    go.Scatter(
        x=y_1d,
        y=np.log(col),
        name=f"{np.round(xlim[0], 2), np.round(xlim[1], 2)} ps",
        line=dict(color=colors[0]),
    )
)

fig.add_trace(go.Scatter(x=y_1d, y=np.log(fit.eval(x=y_1d)), name="fit"))

for model_name, model_value in list(components.items())[1:8]:
    fig.add_annotation(
        x=fit.params[f"{model_name}center"].value,
        y=np.log(fit.eval(x=fit.params[f"{model_name}center"].value)),
        text=f'{fit.params[f"{model_name}center"].value:.2f} ',
        font=dict(size=18, color=colors[0]),
        # ax=-300,
        # ay=0,
        bgcolor="white",
        # opacity=1,
        # bordercolor=colors[0],
        # borderwidth=2,
        # borderpad=4,
    )

fig.update_layout(showlegend=True)

fig.show()

In [ ]:
colors2 = pc.sequential.dense[1::2]

x_plot, y_plot, z_plot = x_s2, y_s2, data_s2

if np.min(x_plot) > 0:
    x_edc = tr_functions.mm_to_ps(x_plot, time_zero)

fits = []
fit_centers = []
fwhms = []
amps = []

xlim1 = (-1, -0.5)
xlim2 = (-0.5, 0)
xlim3 = (0, 0.5)
xlim4 = (0.5, 1)

xlims = [xlim1, xlim2, xlim3, xlim4]
ylim = None

fig = tr_functions.thesis_fig(
    title=f"Fit Component Time Evolution",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

for i, xlim in enumerate(xlims):
    color = colors2[i % len(colors2)]

    y_1d, col = tr_functions.get_1d_x_slice(
        x=x_edc,
        y=y_plot,
        data=z_plot,
        ylims=ylim,
        x_range=xlim,
    )

    fit = fit_FD_7gauss(
        x=y_1d,
        data=col,
        offset_type="constant",
        params=linear_params,
        collect_centers=False,
        fix_centers=True,
        centers=centers,
        plot_fit=False,
    )
    fits.append(fit)

    components = fit.eval_components(x=y_1d)
    for model_name, model_value in list(components.items())[1:8]:
        fig.add_trace(
            go.Scatter(
                x=y_1d,
                y=model_value,
                name=f"{xlim}",
                line_color=color,
                legendgroup=i,
            )
        )
        # if model_name == "iA__":
        #     fig.update_layout(showlegend=True)
        # else:
        #     fig.update_layout(showlegend=False)
        fit_centers.append(fit.params[f"{model_name}center"].value)
        fwhms.append(fit.params[f"{model_name}fwhm"].value)
        amps.append(fit.params[f"{model_name}amplitude"].value)


fig.show()

In [ ]:
import plotly.colors as pc

In [ ]:
xs = [np.mean(vs) for vs in [xlim1, xlim2, xlim3, xlim4]]
colors = pc.qualitative.D3

for param_name in ["amplitude", "height", "fwhm"]:
    # Fig for each fitting parameter
    fig = tr_functions.default_fig()
    fig.update_layout(title=param_name, xaxis_title="delay", yaxis_title=param_name)

    # For each peak individually (one trace per peak)
    for i, letter in enumerate("ABCDEFG"):
        color = colors[i % len(colors)]
        ys = []
        y_errs = []
        y_inits = []
        for fit in fits:
            par = fit.params[f"i{letter}__{param_name}"]
            ys.append(par.value)
            y_errs.append(par.stderr)
            y_inits.append(par.init_value)

        fig.add_trace(
            go.Scatter(
                x=xs,
                y=ys,
                error_y=dict(array=y_errs),
                name="value",
                line=dict(color=color),
                legendgroup=letter,
                legendgrouptitle_text=letter,
                mode="markers+lines",
            )
        )
        # fig.add_trace(
        #     go.Scatter(
        #         x=xs,
        #         y=y_inits,
        #         name="inits",
        #         line=dict(color=color, dash="dash"),
        #         opacity=0.6,
        #         legendgroup=letter,
        #     )
        # )
    fig.show()

# Peak Evolution of CT2

In [ ]:
## Fitting Just TR 4##
title = "Fit of CT2"
x, y, d = x4, y4, d4

## Plot Heatmap Data
fig = tr_functions.thesis_fig(
    title=f"{title}",
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    gridlines=False,
)

x_plot = tr_functions.mm_to_ps(x, time_zero)
y_plot = y

fig.add_trace(go.Heatmap(x=x_plot, y=y_plot, z=d, coloraxis="coloraxis"))
# for h in [2.52, 2.75, 2.96]:
#     fig.add_hline(y=h, line=dict(color="black", width=3, dash="dash"))

fig.show()


## Plot EDC
logplot = False
# logplot = True

if logplot:
    title = f"{title} (logplot)"

xlim = (
    -0.5,
    0.5,
)  # Note: use this to find the center of the state with 1 gauss + offset
# xlim = (-0.75, -0.5)
# xlim = (-0.25, 0.25)
# xlim = (0.25, 0.5)
# xlim = (0.5, 0.75)
# xlim = (0.75, 1)
ylim = None

if np.min(x_plot) > 0:
    x_edc = tr_functions.mm_to_ps(x_plot, time_zero)
else:
    x_edc = x_plot

fig = tr_functions.thesis_fig(
    title=f"{title}",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=x_edc,
    y=y_plot,
    data=d,
    ylims=ylim,
    x_range=xlim,
)

if logplot:
    col = np.log(col)

fig.add_trace(
    go.Scatter(
        x=y_1d,
        y=col,
        name=f"{np.round(xlim[0], 2), np.round(xlim[1], 2)} ps",
        line=dict(color=colors[0]),
    )
)

fig.show()

In [ ]:
x = y_1d
data = col

offset_type = "constant"
plot_fit = True


## Offset
c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

## Gaussians
gauss1 = fitting_functions.make_gaussian(num="A_", amplitude=1, center=2.75, sigma=0.1)
gauss2 = fitting_functions.make_gaussian(num="B_", amplitude=1, center=2.5, sigma=0.1)
gauss3 = fitting_functions.make_gaussian(num="C_", amplitude=1, center=2.95, sigma=0.1)


## Full model
full_model = gauss1 + offset
full_model = gauss1 + gauss2 + gauss3 + offset

params = full_model.make_params()


fit = full_model.fit(data, x=x, params=params)
if plot_fit:
    fit.plot()

print(f"Center A = {fit.params['iA__center'].value:.2f} eV")
print(f"FWHM A = {fit.params['iA__fwhm'].value:.3f} eV")
print(f"Center B = {fit.params['iB__center'].value:.2f} eV")
print(f"FWHM B = {fit.params['iB__fwhm'].value:.3f} eV")
print(f"Center C = {fit.params['iC__center'].value:.2f} eV")
print(f"FWHM C = {fit.params['iC__fwhm'].value:.3f} eV")

In [ ]:
## Plot Data and Fit
fig = tr_functions.thesis_fig(
    title=f"{title}",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
    dtick_y=0.02,
    dtick_x=0.2,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=x_edc,
    y=y_plot,
    data=d,
    ylims=ylim,
    x_range=xlim,
)

fig.add_trace(
    go.Scatter(
        x=y_1d,
        y=col,
        name=f"{np.round(xlim[0], 2), np.round(xlim[1], 2)} ps",
        line=dict(color=colors[0]),
    )
)

fig.add_trace(go.Scatter(x=y_1d, y=fit.eval(x=y_1d), name="fit"))

fig.update_layout(showlegend=True)

## For 1 peak fit:
# for model_name, model_value in list(components.items())[0:1]:
#     fig.add_annotation(
#         x=fit.params[f"{model_name}center"].value,
#         # y=fit.eval(x=fit.params[f"{model_name}center"].value),
#         # xref="x domain",
#         yref="y domain",
#         # The arrow head will be 25% along the x axis, starting from the left
#         # x=0.25,
#         # The arrow head will be 40% along the y axis, starting from the bottom
#         y=0.3,
#         showarrow=False,
#         text=f'Peak center: {fit.params[f"{model_name}center"].value:.2f} eV<br>FWHM: {fit.params[f"{model_name}fwhm"].value:.2f} eV',
#         font=dict(size=18),
#         # ax=-300,
#         # ay=0,
#         bgcolor="white",
#         # opacity=1,
#         # bordercolor=colors[0],
#         # borderwidth=2,
#         # borderpad=4,
#     )

## For 3 peak fit:
for model_name, model_value in list(components.items())[1:4]:
    fig.add_annotation(
        x=fit.params[f"{model_name}center"].value,
        y=fit.eval(x=fit.params[f"{model_name}center"].value),
        text=f'{fit.params[f"{model_name}center"].value:.2f} ',
        font=dict(size=18, color=colors[0]),
        ay=50,
        bgcolor="white",
    )

fig.show()

In [ ]:
## Plot Data and Fit
fig = tr_functions.thesis_fig(
    title=f"{title}",
    xaxis_title=yaxis_title,
    yaxis_title="Residuals",
    equiv_axes=False,
    gridlines=False,
    height=400,
    width=900,
)

fig.add_trace(go.Scatter(x=y_1d, y=col - fit.eval(x=y_1d), name="fit"))
fig.add_hline(y=0, line=dict(dash="dash"))

In [ ]:
## Plot Fit Components ##

fig = tr_functions.thesis_fig(
    title=f"{title}<br> Fit Components",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
)

components = fit.eval_components(x=y_1d)
for model_name, model_value in components.items():
    fig.add_trace(
        go.Scatter(
            x=y_1d,
            y=model_value,
            name=model_name,
        )
    )

fig.data[1].update(name="offset")  # 2 peaks

for model_name, model_value in list(components.items())[0:1]:
    fig.add_annotation(
        x=fit.params[f"{model_name}center"].value,
        # y=fit.eval(x=fit.params[f"{model_name}center"].value),
        # xref="x domain",
        yref="y domain",
        # The arrow head will be 25% along the x axis, starting from the left
        # x=0.25,
        # The arrow head will be 40% along the y axis, starting from the bottom
        y=0.3,
        showarrow=False,
        text=f'Peak center: {fit.params[f"{model_name}center"].value:.2f} eV<br>FWHM: {fit.params[f"{model_name}fwhm"].value:.2f} eV',
        font=dict(size=18),
        # ax=-300,
        # ay=0,
        bgcolor="white",
        # opacity=1,
        # bordercolor=colors[0],
        # borderwidth=2,
        # borderpad=4,
    )

fig.show()

In [ ]:
title = "Fit Component Time Evolution"

offset_type = "constant"

# colors = pc.qualitative.D3
colors = pc.sequential.dense[2:]
# colors = pc.sequential.PuBuGn

# xlim1 = (0, 1)  # zero to 1 ps positive delay

# xlim1 = (np.round(tr_functions.mm_to_ps(np.min(x_plot), time_zero), 2), -0.75)
# xlim2 = (-0.75, -0.5)
# xlim3 = (-0.5, -0.25)
# xlim4 = (-0.25, 0)
# xlim5 = (0, 0.25)
# xlim6 = (0.25, 0.5)
# xlim7 = (0.5, 0.75)
# xlim8 = (0.75, 1)

# xlims = [xlim1, xlim2, xlim3, xlim4, xlim5, xlim6, xlim7, xlim8]

xlim1 = (np.round(tr_functions.mm_to_ps(np.min(x_plot), time_zero), 2), -0.5)
xlim2 = (-0.5, 0)
xlim3 = (0, 0.5)
xlim4 = (0.5, 1)

xlims = [xlim1, xlim2, xlim3, xlim4]


ylim = (np.min(y_plot), 2.3)

## Get and plot data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title="Energy (eV)",
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

centers_FD = []
temps_FD = []
centers_gaussA = []
centers_gaussB = []
fwhm_gaussA = []
fwhm_gaussB = []
amp_gaussB = []


for i, xlim in enumerate(xlims):
    color = colors[i % len(colors)]

    _, y_1d, col = plot_EDC(
        x_plot, y_plot, z_plot, title=title, xlim=xlim, ylim=ylim, show_plot=False
    )

    if offset_type == "linear":
        fit = fit_FD_and_peaks(
            xaxis=y_1d, data=col, offset_type=offset_type, plot_fit=False
        )
    elif offset_type == "constant":
        fit = fit_FD_and_peaks(
            xaxis=y_1d, data=col, offset_type="linear", plot_fit=False
        )
        linear_params = fit.params.copy()

        fit = fit_FD_and_peaks(
            xaxis=y_1d,
            data=col,
            params=linear_params,
            offset_type="constant",
            plot_fit=False,
        )

    components = fit.eval_components(x=y_1d)
    for model_name, model_value in list(components.items())[2:3]:
        fig.add_trace(
            go.Scatter(
                x=y_1d,
                y=model_value,
                name=f"{model_name}{xlim}",
                line_color=color,
            )
        )

    if model_name == "fermi_dirac":
        fig.add_annotation(
            # x=fit.params[f"center"].value,
            # y=fit.params[f"amp"].value / 2,
            xref="paper",
            yref="paper",
            x=0.1,
            y=fit.params[f"amp"].value / 2 + i / 10 - 0.5,
            showarrow=False,
            text=f'FD {i+1}: {fit.params[f"center"].value:.2f} eV, {fit.params[f"theta"].value/k_B:.2f} K',
            font=dict(size=16, color=color),
            # ax=-300,
            # ay=0,
            bgcolor="white",
            # opacity=1,
            bordercolor=color,
            borderwidth=2,
            borderpad=4,
        )
    elif model_name == "iB__":
        fig.add_annotation(
            x=fit.params[f"{model_name}center"].value,
            y=fit.params[f"{model_name}height"].value,
            text=f'Peak {i+1}: {fit.params[f"{model_name}center"].value:.2f} eV, {fit.params[f"{model_name}fwhm"].value:.2f} ps',
            font=dict(size=16, color=color),
            ax=-300,
            ay=0,
            bgcolor="white",
            # opacity=1,
            bordercolor=color,
            borderwidth=2,
            borderpad=4,
        )
    else:
        fig.add_annotation(
            x=fit.params[f"{model_name}center"].value,
            y=fit.params[f"{model_name}height"].value,
            text=f'Peak {i+1}: {fit.params[f"{model_name}center"].value:.2f} eV, {fit.params[f"{model_name}fwhm"].value:.2f} ps',
            font=dict(size=16, color=color),
            ax=300,
            ay=(i - 3) * -60,
            # ay=fit.params[f"{model_name}height"].value / 2 + i * 90,
            bgcolor="white",
            # opacity=1,
            bordercolor=color,
            borderwidth=2,
            borderpad=4,
        )

    centers_FD.append(fit.params["center"].value)
    temps_FD.append(fit.params["theta"].value / k_B)
    centers_gaussA.append(fit.params["iA__center"].value)
    centers_gaussB.append(fit.params["iB__center"].value)
    fwhm_gaussA.append(fit.params["iA__fwhm"].value)
    fwhm_gaussB.append(fit.params["iB__fwhm"].value)
    amp_gaussB.append(fit.params["iB__amplitude"].value)

    # print(f"limit: {xlim} ps")
    # print(f"Center FD = {fit.params['center'].value:.2f} eV")
    # print(f"T = {fit.params['theta'].value / k_B:.2f} K")
    # print(f"Center = {fit.params[f'{model_name}center'].value:.2f} eV")
    # print(f"FWHM = {fit.params[f'{model_name}fwhm'].value:.3f} ps")

fig.show()